In [2]:
import pandas as pd
import os
import seaborn as sns
from skbio.diversity import alpha_diversity
from skbio.diversity import beta_diversity
from skbio import TreeNode
from io import StringIO
from skbio.stats.ordination import pcoa

import cx_Oracle

SERVER = 'dboracle.itk.ppke.hu'
PORT = 1521
SERVICE = 'rsc.itk.ppke.hu'
USER = 'metagenome'
PASSWORD = 'LrJPRUS73r84'

class Connection():

    def __init__(self):
        dsn_tns = cx_Oracle.makedsn(SERVER, PORT, service_name=SERVICE)
        self.connection = cx_Oracle.connect(USER, PASSWORD, dsn_tns)

    def get_connection(self):
        return self.connection

    def select(self, sql):
        return pd.read_sql(sql, con=self.connection)

con = Connection()

In [3]:
path = '//gfs/data/curated_metagenomes_metaphlan2/'

files = {}
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        for r1, d1, f1 in os.walk(os.path.join(path,directory)):
            for file in f1:
                if '.profile.txt' in file:
                    if directory in files:
                        files[directory].append(os.path.join(r1, file))
                    else:
                        files[directory] = [(os.path.join(r1, file))]

In [4]:
def get_data_for_sample(path_to_file):
    dataset = path_to_file.split('/')[-2]
    sample_id = path_to_file.split('/')[-1].split('.')[0]
    
    df = pd.read_csv(path_to_file, delimiter = '\t')
    
    profile_df_2 = df
    profile_df_2['Rank'] = None
    profile_df_2['K'] = None
    profile_df_2['P'] = None
    profile_df_2['C'] = None
    profile_df_2['O'] = None
    profile_df_2['F'] = None
    profile_df_2['G'] = None
    profile_df_2['S'] = None
    profile_df_2['T'] = None
    profile_df_2['Abundance'] = profile_df_2["Metaphlan2_Analysis"]
    
    for p in profile_df_2["#SampleID"]:
        row_level = None
        tags = p.split('|')
        for tag in tags:
            tmp = tag.split('__')
            try:
                level = tmp[0]
                name = tmp[1]
                profile_df_2
                row_level = level

                profile_df_2.loc[profile_df_2['#SampleID'] == p, level] = name
            except: 
                print("Can not split:", tmp)
        profile_df_2.loc[profile_df_2['#SampleID'] == p, 'Rank'] = row_level
    
    def get_name_for_row(row):
        try:
            return row[row.Rank]
        except:
            return '???'
    
    profile_df_2["Name"] = profile_df_2.apply(get_name_for_row, axis=1)

    selected = profile_df_2[["Rank", "Name", "Abundance"]]
    
    selected['Sample_ID'] = sample_id
    selected['Dataset'] = dataset

    
    return selected[["Dataset", "Sample_ID", "Rank", "Name", "Abundance"]]

In [5]:
def join_metadata(df, dataset_name):
    SQL_METADATA = """
        select
            DATASET_NAME,
            STUDY_CONDITION,
            BODY_SITE,
            SUBJECTID,
            NCBI_ACCESSION,
            COUNTRY,
            DISEASE,
            GENDER,
            AGE,
            AGE_CATEGORY
        from METADATA
        where dataset_name = '{dataset_name}'
        """.replace("{dataset_name}", dataset_name)
    df_metadata = con.select(SQL_METADATA)
    
    df["DATASET_NAME"] = df["Dataset"]
    merged = df.merge(df_metadata, on='DATASET_NAME')
    
    col_to_idx = {}
    for i in range(len(merged.columns)):
        col_to_idx[merged.columns[i]] = i
    
    joined = merged.loc[[row[1][col_to_idx["Sample_ID"]] in row[1][col_to_idx["NCBI_ACCESSION"]] 
                        for row in merged.iterrows()]]
    
    output = joined[["Sample_ID", "Dataset", "Rank", "Name", "Abundance", 
                     "STUDY_CONDITION", "BODY_SITE", "SUBJECTID",
                     "COUNTRY", "DISEASE", "AGE", "AGE_CATEGORY"]]
    output = output.reset_index()
    output = output.drop(columns="index")
    return output

In [ ]:
dataset_dfs = {}
i = 0;
for dataset_name, filelist in files.items():
    if dataset_name == "YuJ_2015":
        df_list = []
        fc = 0
        for file in filelist:
            print(file)
            df = get_data_for_sample(file)
            if df is not None:
                df_list.append(get_data_for_sample(file))
            fc = fc+1
        df_dataset = pd.concat(df_list)
        output = join_metadata(df_dataset, dataset_name)
        print("Files in dataset:", fc)
        
        path_to_output = os.path.join(path, dataset_name + ".csv")
        with open(path_to_output, "w") as f: 
            f.write(output.to_csv()) 
    i = i+1
    print(dataset_name, " kész.")

AsnicarF_2017  kész.
ChngKR_2016  kész.
FengQ_2015  kész.
Heitz-BuschartA_2016  kész.
KarlssonFH_2013  kész.
LeChatelierE_2013  kész.
LiuW_2016  kész.
LomanNJ_2013  kész.
Obregon-TitoAJ_2015  kész.
QinJ_2012  kész.
QinN_2014  kész.
RampelliS_2015  kész.
SchirmerM_2016  kész.
VatanenT_2016  kész.
VogtmannE_2016  kész.
XieH_2016  kész.
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018191.profile.txt


/gfs/progs/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/gfs/progs/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018205.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018223.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018198.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018212.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018199.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018206.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018213.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018192.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018185.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018200.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018214.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018207.profile.txt
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018193.profile.txt
//gfs/data/curated_metagenomes_metaphl

In [120]:
len(files['YuJ_2015'])

128

In [34]:
path_to_file = files["AsnicarF_2017"][0]

In [36]:
path_to_file

'//gfs/data/curated_metagenomes_metaphlan2/AsnicarF_2017/SRR4052021.profile.txt'

In [66]:
df = get_data_for_sample(path_to_file)

                       #SampleID  Metaphlan2_Analysis Rank         k  \
0                    k__Bacteria             97.92389    k  Bacteria   
1                     k__Viruses              2.07611    k   Viruses   
2  k__Bacteria|p__Proteobacteria             78.59277    p  Bacteria   
3  k__Bacteria|p__Actinobacteria             12.60032    p  Bacteria   
4      k__Bacteria|p__Firmicutes              6.73081    p  Bacteria   

                p     c     o     f     g     s     t  Abundance  
0            None  None  None  None  None  None  None   97.92389  
1            None  None  None  None  None  None  None    2.07611  
2  Proteobacteria  None  None  None  None  None  None   78.59277  
3  Actinobacteria  None  None  None  None  None  None   12.60032  
4      Firmicutes  None  None  None  None  None  None    6.73081  


/gfs/progs/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/gfs/progs/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [67]:
df.head()

,Dataset,Sample_ID,Rank,Name,Abundance
0,AsnicarF_2017,SRR4052021,k,Bacteria,97.92389
1,AsnicarF_2017,SRR4052021,k,Viruses,2.07611
2,AsnicarF_2017,SRR4052021,p,Proteobacteria,78.59277
3,AsnicarF_2017,SRR4052021,p,Actinobacteria,12.60032
4,AsnicarF_2017,SRR4052021,p,Firmicutes,6.73081


In [68]:
joined = join_metadata(df, "AsnicarF_2017")

In [69]:
joined.head()

,Sample_ID,Dataset,Rank,Name,Abundance,STUDY_CONDITION,BODY_SITE,SUBJECTID,COUNTRY,DISEASE,AGE,AGE_CATEGORY
0,SRR4052021,AsnicarF_2017,k,Bacteria,97.92389,control,stool,MV_FEI1,ITA,healthy,1.0,newborn
1,SRR4052021,AsnicarF_2017,k,Viruses,2.07611,control,stool,MV_FEI1,ITA,healthy,1.0,newborn
2,SRR4052021,AsnicarF_2017,p,Proteobacteria,78.59277,control,stool,MV_FEI1,ITA,healthy,1.0,newborn
3,SRR4052021,AsnicarF_2017,p,Actinobacteria,12.60032,control,stool,MV_FEI1,ITA,healthy,1.0,newborn
4,SRR4052021,AsnicarF_2017,p,Firmicutes,6.73081,control,stool,MV_FEI1,ITA,healthy,1.0,newborn


In [ ]:
profile_df = pd.read_csv(path_to_profile, delimiter = '\t', skiprows = [1])

#f = open("/gfs/data/curated_metagenomes_kraken2/RampelliS_2015⁩/profiled_metagenome.txt", 
#encoding="utf-8")

In [9]:
files.keys()

dict_keys(['AsnicarF_2017', 'ChngKR_2016', 'FengQ_2015', 'Heitz-BuschartA_2016', 'KarlssonFH_2013', 'LeChatelierE_2013', 'LiuW_2016', 'LomanNJ_2013', 'Obregon-TitoAJ_2015', 'QinJ_2012', 'QinN_2014', 'RampelliS_2015', 'SchirmerM_2016', 'VatanenT_2016', 'VogtmannE_2016', 'XieH_2016', 'YuJ_2015', 'ZellerG_2014'])

In [11]:
path_to_file = files["AsnicarF_2017"][0]

In [12]:
dataset = path_to_file.split('/')[-2]
df = pd.read_csv(path_to_file, delimiter = '\t')


In [13]:
df.head()

,#SampleID,Metaphlan2_Analysis
0,k__Bacteria,97.92389
1,k__Viruses,2.07611
2,k__Bacteria|p__Proteobacteria,78.59277
3,k__Bacteria|p__Actinobacteria,12.60032
4,k__Bacteria|p__Firmicutes,6.73081


In [14]:
profile_df_2 = df
profile_df_2['level'] = None
profile_df_2['k'] = None
profile_df_2['p'] = None
profile_df_2['c'] = None
profile_df_2['o'] = None
profile_df_2['f'] = None
profile_df_2['g'] = None
profile_df_2['s'] = None
profile_df_2['t'] = None
profile_df_2['value'] = profile_df_2["Metaphlan2_Analysis"]

In [15]:
profile_df_2.head()

,#SampleID,Metaphlan2_Analysis,level,k,p,c,o,f,g,s,t,value
0,k__Bacteria,97.92389,None,None,None,None,None,None,None,None,None,97.92389
1,k__Viruses,2.07611,None,None,None,None,None,None,None,None,None,2.07611
2,k__Bacteria|p__Proteobacteria,78.59277,None,None,None,None,None,None,None,None,None,78.59277
3,k__Bacteria|p__Actinobacteria,12.60032,None,None,None,None,None,None,None,None,None,12.60032
4,k__Bacteria|p__Firmicutes,6.73081,None,None,None,None,None,None,None,None,None,6.73081


In [16]:
for p in profile_df_2["#SampleID"]:
    row_level = None
    tags = p.split('|')
    for tag in tags:
        tmp = tag.split('__')
        level = tmp[0]
        name = tmp[1]
        profile_df_2
        row_level = level
        
        profile_df_2.loc[profile_df_2['#SampleID'] == p, level] = name
    profile_df_2.loc[profile_df_2['#SampleID'] == p, 'level'] = row_level

In [17]:
profile_df_2.head()

,#SampleID,Metaphlan2_Analysis,level,k,p,c,o,f,g,s,t,value
0,k__Bacteria,97.92389,k,Bacteria,None,None,None,None,None,None,None,97.92389
1,k__Viruses,2.07611,k,Viruses,None,None,None,None,None,None,None,2.07611
2,k__Bacteria|p__Proteobacteria,78.59277,p,Bacteria,Proteobacteria,None,None,None,None,None,None,78.59277
3,k__Bacteria|p__Actinobacteria,12.60032,p,Bacteria,Actinobacteria,None,None,None,None,None,None,12.60032
4,k__Bacteria|p__Firmicutes,6.73081,p,Bacteria,Firmicutes,None,None,None,None,None,None,6.73081


In [22]:
profile_df_2["Name"] = [profile_df_2[i] for i in profile_df_2["level"]]

In [26]:
profile_df_2["Name"]  = profile_df_2.apply(lambda row: row[row.level], axis=1)

In [29]:
selected = profile_df_2[["level", "Name", "value"]]

In [30]:
selected.head()

,level,Name,value
0,k,Bacteria,97.92389
1,k,Viruses,2.07611
2,p,Proteobacteria,78.59277
3,p,Actinobacteria,12.60032
4,p,Firmicutes,6.73081


In [101]:
b = pd.read_csv('/gfs/data/curated_metagenomes_metaphlan2/ChngKR_2016/SRR1950687.bowtie2out.txt')

In [103]:
b.shape

(1316718, 1)

In [106]:
(2407746+2408960)

4816706

In [107]:
1316718/4816706*100

27.33648265017628

In [ ]:
b = pd.read_csv('/gfs/data/curated_metagenomes_metaphlan2/ChngKR_2016/SRR1950687.bowtie2out.txt')

In [105]:
p = pd.read_csv('/gfs/data/curated_metagenomes_preprocessed/ChngKR_2016/trimmed_files/SRR1950687_1.fastq.trimmed.gz')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [123]:
dataset_dfs = {}
i = 0;
for dataset_name, filelist in files.items():
    if dataset_name == "YuJ_2015":
        df_list = []
        fc = 0
        for file in filelist:
            print(file)
            df = get_data_for_sample(file)
            print(df.head())
            if df is not None:
                df_list.append(get_data_for_sample(file))
            fc = fc+1
        df_dataset = pd.concat(df_list)
        output = join_metadata(df_dataset, dataset_name)
        print("Files in dataset:", fc)
        
        path_to_output = os.path.join(path, dataset_name + ".csv")
        with open(path_to_output, "w") as f: 
            f.write(output.to_csv()) 
    i = i+1
    print(dataset_name, " kész.")

AsnicarF_2017  kész.
ChngKR_2016  kész.
FengQ_2015  kész.
Heitz-BuschartA_2016  kész.
KarlssonFH_2013  kész.
LeChatelierE_2013  kész.
LiuW_2016  kész.
LomanNJ_2013  kész.
Obregon-TitoAJ_2015  kész.
QinJ_2012  kész.
QinN_2014  kész.
RampelliS_2015  kész.
SchirmerM_2016  kész.
VatanenT_2016  kész.
VogtmannE_2016  kész.
XieH_2016  kész.
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018191.profile.txt


/gfs/progs/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/gfs/progs/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


    Dataset   Sample_ID Rank            Name  Abundance
0  YuJ_2015  ERR1018191    k        Bacteria   99.98996
1  YuJ_2015  ERR1018191    k         Viruses    0.01004
2  YuJ_2015  ERR1018191    p      Firmicutes   58.59706
3  YuJ_2015  ERR1018191    p  Actinobacteria   18.78107
4  YuJ_2015  ERR1018191    p  Proteobacteria   18.20503
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018205.profile.txt
    Dataset   Sample_ID Rank            Name  Abundance
0  YuJ_2015  ERR1018205    k        Bacteria   99.91806
1  YuJ_2015  ERR1018205    k       Eukaryota    0.08194
2  YuJ_2015  ERR1018205    p      Firmicutes   41.98932
3  YuJ_2015  ERR1018205    p   Bacteroidetes   36.04325
4  YuJ_2015  ERR1018205    p  Actinobacteria   17.76978
//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018223.profile.txt
    Dataset   Sample_ID Rank            Name  Abundance
0  YuJ_2015  ERR1018223    k        Bacteria   99.60468
1  YuJ_2015  ERR1018223    k         Viruses    0.39532
2  YuJ_2015 

In [124]:
d

[]

In [10]:
import pandas as pd
import os
import seaborn as sns
from skbio.diversity import alpha_diversity
from skbio.diversity import beta_diversity
from skbio import TreeNode
from io import StringIO
from skbio.stats.ordination import pcoa

import cx_Oracle

SERVER = 'dboracle.itk.ppke.hu'
PORT = 1521
SERVICE = 'rsc.itk.ppke.hu'
USER = 'metagenome'
PASSWORD = 'LrJPRUS73r84'

class Connection():

    def __init__(self):
        dsn_tns = cx_Oracle.makedsn(SERVER, PORT, service_name=SERVICE)
        self.connection = cx_Oracle.connect(USER, PASSWORD, dsn_tns)

    def get_connection(self):
        return self.connection

    def select(self, sql):
        return pd.read_sql(sql, con=self.connection)

con = Connection()

path = '//gfs/data/curated_metagenomes_metaphlan2/'

files = {}
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        for r1, d1, f1 in os.walk(os.path.join(path,directory)):
            for file in f1:
                if '.profile.txt' in file:
                    if directory in files:
                        files[directory].append(os.path.join(r1, file))
                    else:
                        files[directory] = [(os.path.join(r1, file))]

def get_data_for_sample(path_to_file):
    dataset = path_to_file.split('/')[-2]
    sample_id = path_to_file.split('/')[-1].split('.')[0]

    df = pd.read_csv(path_to_file, delimiter = '\t')

    profile_df_2 = df
    profile_df_2['Rank'] = None
    profile_df_2['K'] = None
    profile_df_2['P'] = None
    profile_df_2['C'] = None
    profile_df_2['O'] = None
    profile_df_2['F'] = None
    profile_df_2['G'] = None
    profile_df_2['S'] = None
    profile_df_2['T'] = None
    profile_df_2['Abundance'] = profile_df_2["Metaphlan2_Analysis"]

    for p in profile_df_2["#SampleID"]:
        row_level = None
        tags = p.split('|')
        for tag in tags:
            tmp = tag.split('__')
            try:
                level = tmp[0]
                name = tmp[1]
                profile_df_2
                row_level = level

                profile_df_2.loc[profile_df_2['#SampleID'] == p, level] = name
            except:
                print("Can not split:", tmp)
        profile_df_2.loc[profile_df_2['#SampleID'] == p, 'Rank'] = row_level

    def get_name_for_row(row):
        try:
            return row[row.Rank]
        except:
            return '???'

    profile_df_2["Name"] = profile_df_2.apply(get_name_for_row, axis=1)

    selected = profile_df_2[["Rank", "Name", "Abundance"]]

    selected['Sample_ID'] = sample_id
    selected['Dataset'] = dataset


    return selected[["Dataset", "Sample_ID", "Rank", "Name", "Abundance"]]

def join_metadata(df, dataset_name):
    SQL_METADATA = """
        select
            DATASET_NAME,
            STUDY_CONDITION,
            BODY_SITE,
            SUBJECTID,
            NCBI_ACCESSION,
            COUNTRY,
            DISEASE,
            GENDER,
            AGE,
            AGE_CATEGORY
        from METADATA
        where dataset_name = '{dataset_name}'
        """.replace("{dataset_name}", dataset_name)
    df_metadata = con.select(SQL_METADATA)

    df["DATASET_NAME"] = df["Dataset"]
    merged = df.merge(df_metadata, on='DATASET_NAME')

    col_to_idx = {}
    for i in range(len(merged.columns)):
        col_to_idx[merged.columns[i]] = i

    joined = merged.loc[[row[1][col_to_idx["Sample_ID"]] in row[1][col_to_idx["NCBI_ACCESSION"]]
                        for row in merged.iterrows()]]

    output = joined[["Sample_ID", "Dataset", "Rank", "Name", "Abundance",
                     "STUDY_CONDITION", "BODY_SITE", "SUBJECTID",
                     "COUNTRY", "DISEASE", "AGE", "AGE_CATEGORY"]]
    output = output.reset_index()
    output = output.drop(columns="index")
    return output

In [18]:
files["YuJ_2015"][0]

'//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018191.profile.txt'

In [16]:
df = get_data_for_sample('//gfs/data/curated_metagenomes_metaphlan2/YuJ_2015/ERR1018191.profile.txt')

/gfs/progs/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/gfs/progs/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
df.head()

,Dataset,Sample_ID,Rank,Name,Abundance
0,YuJ_2015,ERR1018191,k,Bacteria,99.98996
1,YuJ_2015,ERR1018191,k,Viruses,0.01004
2,YuJ_2015,ERR1018191,p,Firmicutes,58.59706
3,YuJ_2015,ERR1018191,p,Actinobacteria,18.78107
4,YuJ_2015,ERR1018191,p,Proteobacteria,18.20503


In [20]:
joined = join_metadata(df, "YuJ_2015")

,Sample_ID,Dataset,Rank,Name,Abundance,STUDY_CONDITION,BODY_SITE,SUBJECTID,COUNTRY,DISEASE,AGE,AGE_CATEGORY


In [28]:
SQL_METADATA = """
    select
        DATASET_NAME,
        STUDY_CONDITION,
        BODY_SITE,
        SUBJECTID,
        NCBI_ACCESSION,
        COUNTRY,
        DISEASE,
        GENDER,
        AGE,
        AGE_CATEGORY
    from METADATA
    where dataset_name = '{dataset_name}'
    """.replace("{dataset_name}", "YuJ_2015")
df_metadata = con.select(SQL_METADATA)

In [29]:
df_metadata.head()

,DATASET_NAME,STUDY_CONDITION,BODY_SITE,SUBJECTID,NCBI_ACCESSION,COUNTRY,DISEASE,GENDER,AGE,AGE_CATEGORY
0,YuJ_2015,CRC,stool,SZAXPI017595-169,ERS848745,CHN,CRC,None,None,senior
1,YuJ_2015,CRC,stool,SZAXPI017594-166,ERS848744,CHN,CRC,None,None,senior
2,YuJ_2015,CRC,stool,SZAXPI017593-158,ERS848743,CHN,CRC,None,None,senior
3,YuJ_2015,control,stool,SZAXPI017592-142,ERS848742,CHN,none,None,None,senior
4,YuJ_2015,control,stool,SZAXPI017591-140,ERS848741,CHN,none,None,None,senior


In [30]:
df["DATASET_NAME"] = df["Dataset"]
merged = df.merge(df_metadata, on='DATASET_NAME')

In [32]:
merged.head()

,Dataset,Sample_ID,Rank,Name,Abundance,DATASET_NAME,STUDY_CONDITION,BODY_SITE,SUBJECTID,NCBI_ACCESSION,COUNTRY,DISEASE,GENDER,AGE,AGE_CATEGORY
0,YuJ_2015,ERR1018191,k,Bacteria,99.98996,YuJ_2015,CRC,stool,SZAXPI017595-169,ERS848745,CHN,CRC,None,None,senior
1,YuJ_2015,ERR1018191,k,Bacteria,99.98996,YuJ_2015,CRC,stool,SZAXPI017594-166,ERS848744,CHN,CRC,None,None,senior
2,YuJ_2015,ERR1018191,k,Bacteria,99.98996,YuJ_2015,CRC,stool,SZAXPI017593-158,ERS848743,CHN,CRC,None,None,senior
3,YuJ_2015,ERR1018191,k,Bacteria,99.98996,YuJ_2015,control,stool,SZAXPI017592-142,ERS848742,CHN,none,None,None,senior
4,YuJ_2015,ERR1018191,k,Bacteria,99.98996,YuJ_2015,control,stool,SZAXPI017591-140,ERS848741,CHN,none,None,None,senior


In [33]:
col_to_idx = {}
for i in range(len(merged.columns)):
    col_to_idx[merged.columns[i]] = i

In [36]:
col_to_idx

{'Dataset': 0,
 'Sample_ID': 1,
 'Rank': 2,
 'Name': 3,
 'Abundance': 4,
 'DATASET_NAME': 5,
 'STUDY_CONDITION': 6,
 'BODY_SITE': 7,
 'SUBJECTID': 8,
 'NCBI_ACCESSION': 9,
 'COUNTRY': 10,
 'DISEASE': 11,
 'GENDER': 12,
 'AGE': 13,
 'AGE_CATEGORY': 14}

In [39]:
row = [row for row in merged.iterrows()][0]

In [44]:
row = row[1]

In [45]:
row

Dataset                    YuJ_2015
Sample_ID                ERR1018191
Rank                              k
Name                       Bacteria
Abundance                     99.99
DATASET_NAME               YuJ_2015
STUDY_CONDITION                 CRC
BODY_SITE                     stool
SUBJECTID          SZAXPI017595-169
NCBI_ACCESSION            ERS848745
COUNTRY                         CHN
DISEASE                         CRC
GENDER                         None
AGE                            None
AGE_CATEGORY                 senior
Name: 0, dtype: object

In [46]:
row[col_to_idx["Sample_ID"]]

'ERR1018191'

In [47]:
row[col_to_idx["NCBI_ACCESSION"]]

'ERS848745'

In [34]:
joined = merged.loc[[row[1][col_to_idx["Sample_ID"]] in row[1][col_to_idx["NCBI_ACCESSION"]]
                    for row in merged.iterrows()]]

In [35]:
joined.head()

,Dataset,Sample_ID,Rank,Name,Abundance,DATASET_NAME,STUDY_CONDITION,BODY_SITE,SUBJECTID,NCBI_ACCESSION,COUNTRY,DISEASE,GENDER,AGE,AGE_CATEGORY


In [ ]:

dataset_dfs = {}
i = 0;
for dataset_name, filelist in files.items():
    if dataset_name == "YuJ_2015":
        df_list = []
        for file in filelist:
            print(file)
            df = get_data_for_sample(file)
            if df is not None:
                df_list.append(get_data_for_sample(file))
        df_dataset = pd.concat(df_list)
        output = join_metadata(df_dataset, dataset_name)

        path_to_output = os.path.join(path, dataset_name + ".csv")
        with open(path_to_output, "w") as f:
            f.write(output.to_csv())
    i = i+1
    print(dataset_name, " kesz.")

